- **Pydantic** is a Python library for data validation and settings management using Python type annotations.
- It is build by rust so its super fast and it performs validation and type conversion super easy and faster.
- It ensures that data types are correct by parsing and validating input data (e.g., from APIs or user input).
- Built on Python’s `dataclasses` and `typing` modules for cleaner, more readable code.
- Automatically raises errors if input data doesn't conform to expected types or constraints.
- Commonly used with **FastAPI**, **ORMs**, and **configuration management**.


In [ ]:
from dataclasses import dataclass

@dataclass
class Product:
    name: str
    price: float
    in_stock: bool = True

item = Product("Chocolate", 2.99)
print(item)

#data validation is not done automatically in dataclasses
item2 = Product("Chocolate", '222')
print(item2)

Product(name='Chocolate', price=2.99, in_stock=True)
Product(name='Chocolate', price='222', in_stock=True)


In [9]:
from pydantic import BaseModel, ValidationError
from typing import Optional,List


class Product(BaseModel):
    name: str
    price: float
    stock_list : Optional[List[str]] = []
    in_stock: Optional[bool] = False  # Now it's optional

item = Product(name="Chocolate", price=2.99,stock_list=["item1", "item2"])
print(item)

item1 = Product(name="Chocolate", price=2.99,in_stock=True,stock_list=("item1", "item2",1))
print(item1)


name='Chocolate' price=2.99 stock_list=['item1', 'item2'] in_stock=False


ValidationError: 1 validation error for Product
stock_list.2
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [10]:
item = Product(name="Chocolate", price="2",in_stock=False)
print(item)


name='Chocolate' price=2.0 stock_list=[] in_stock=False


In [11]:
from pydantic import BaseModel, StrictFloat

class Product(BaseModel):
    name: str
    price: StrictFloat
    in_stock: bool = True

item = Product(name="Chocolate", price=2.99)
print(item)


name='Chocolate' price=2.99 in_stock=True


In [12]:
item = Product(name="Chocolate", price="2",in_stock=False)
print(item)


ValidationError: 1 validation error for Product
price
  Input should be a valid number [type=float_type, input_value='2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_type

In [13]:
from pydantic import BaseModel
from typing import Optional

class Category(BaseModel):
    id: int
    name: str

class Product(BaseModel):
    name: str
    price: float
    in_stock: Optional[bool] = True
    category: Category  # Nested model

# Creating a product with a nested category
item = Product(
    name="Chocolate",
    price=2.99,
    category=Category(id=1, name="Snacks")
)

print(item)


name='Chocolate' price=2.99 in_stock=True category=Category(id=1, name='Snacks')


In [14]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=2, max_length=50, description="Product name")
    price: float = Field(..., gt=0, description="Must be greater than 0")
    stock: int = Field(default=0, ge=0, description="Non-negative integer")


In [15]:
product = Product(name="Apple", price=1.25)
print(product)


name='Apple' price=1.25 stock=0


In [16]:
product.model_json_schema()

{'properties': {'name': {'description': 'Product name',
   'maxLength': 50,
   'minLength': 2,
   'title': 'Name',
   'type': 'string'},
  'price': {'description': 'Must be greater than 0',
   'exclusiveMinimum': 0,
   'title': 'Price',
   'type': 'number'},
  'stock': {'default': 0,
   'description': 'Non-negative integer',
   'minimum': 0,
   'title': 'Stock',
   'type': 'integer'}},
 'required': ['name', 'price'],
 'title': 'Product',
 'type': 'object'}

In [17]:
from pydantic import BaseModel, Field
from typing import List, Dict
from datetime import datetime
import uuid

class Example(BaseModel):
    # ❌ BAD: Same list shared between instances
    static_list: List[str] = []  

    # ✅ GOOD: New list per instance
    dynamic_list: List[str] = Field(default_factory=list)

    # Static default value (string)
    status: str = "pending"

    # Dynamic default value: UUID
    session_id: str = Field(default_factory=lambda: str(uuid.uuid4()))

    # Dynamic default value: current time
    created_at: datetime = Field(default_factory=datetime.utcnow)

# Create two instances
ex1 = Example()
ex2 = Example()

# Modify the static list in one instance
ex1.static_list.append(1)

print("Ex1:", id(ex1.static_list))
print("Ex2:", id(ex2.static_list))  

print("Ex1 dynamic_list:", ex1.dynamic_list)  # [] ✅
print("Ex2 dynamic_list:", ex2.dynamic_list)  # [] ✅

print("Ex1 UUID:", ex1.session_id)
print("Ex2 UUID:", ex2.session_id)  # Different ✅

print("Ex1 time:", ex1.created_at)
print("Ex2 time:", ex2.created_at)  # Different (created separately) ✅


Ex1: 1519128949952
Ex2: 1519128940544
Ex1 dynamic_list: []
Ex2 dynamic_list: []
Ex1 UUID: a128ac21-7231-4815-bcc3-949dfd0935da
Ex2 UUID: b976dfe5-cd43-4229-a4ca-282594c921c1
Ex1 time: 2025-05-24 05:24:49.844873
Ex2 time: 2025-05-24 05:24:49.844873


In [18]:
from pydantic import BaseModel, field_validator

class User(BaseModel):
    age: int

    @field_validator('age')
    def age_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Age must be positive')
        return v
user = User(age=25)
print(user)


age=25


In [19]:
user = User(age=-25)
print(user)

ValidationError: 1 validation error for User
age
  Value error, Age must be positive [type=value_error, input_value=-25, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [20]:
from pydantic import BaseModel, model_validator
from datetime import date

class Booking(BaseModel):
    start_date: date
    end_date: date

    @model_validator(mode='after')
    def check_dates(self):
        if self.end_date < self.start_date:
            raise ValueError('End date must be after start date')
        return self


# -----------------------
# Valid Input
# -----------------------
try:
    booking = Booking(start_date='2025-05-01', end_date='2025-05-10')
    print("Valid booking:", booking)
except ValidationError as e:
    print("Validation Error:\n", e)

# -----------------------
# Invalid Input
# -----------------------
try:
    booking = Booking(start_date='2025-05-10', end_date='2025-05-01')
    print("Booking created:", booking)
except ValidationError as e:
    print("Validation Error:\n", e)


Valid booking: start_date=datetime.date(2025, 5, 1) end_date=datetime.date(2025, 5, 10)
Validation Error:
 1 validation error for Booking
  Value error, End date must be after start date [type=value_error, input_value={'start_date': '2025-05-1...end_date': '2025-05-01'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error
